In [2]:
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("../")
from data.occnet_data_loader import OccnetTfrecordLoader
from network.main import Transformer

In [3]:
import cv2
import math

In [4]:
sess = tf.Session()

In [5]:
vw, vh = 128, 128

In [6]:
occnet_dataloader = OccnetTfrecordLoader("../datasets/000/", occnet_data=True)
occnet_transformer = Transformer(128, 128, "../datasets/000/", occnet=True)
occnet_features = occnet_dataloader.get_features()
# features = sess.run(features)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [7]:
def func1(x):
    return tf.transpose(tf.reshape(occnet_features[x], [-1, 4, 4]), [0, 2, 1])

mv = [func1("mv%d" % i) for i in range(2)]
mvi = [func1("mvi%d" % i) for i in range(2)]
img0_depth = tf.reshape(occnet_features["img0_depth"], [-1, 480, 640])

u, v = 0.0, 0.0
points = []
for z in np.linspace(-0.5, 0.5, num=100):
    z_value = (img0_depth[0, 240, 320] / 1000.0) + z
    points.append([u, v, z_value])

# add back batch dimension
uvz = tf.expand_dims(tf.convert_to_tensor(points), axis=0)

world_coords = tf.matmul(occnet_transformer.unproject(uvz), mvi[0])
uvz_proj = occnet_transformer.project(tf.matmul(world_coords, mv[1 - 0]))

uvz_np, uvz_proj_np, img0_np, img1_np, img0_depth_np = sess.run(
    [
        uvz,
        uvz_proj,
        occnet_features["img0"][0, :, :, :3],
        occnet_features["img1"][0, :, :, :3],
        img0_depth
    ]
)

In [8]:
# a = tf.constant([0.08291021, -0.00866176,  0.549])
# b = tf.constant([0.04130268, 0.00666338, 0.543])

a = tf.constant([0.13739401, -0.06075361,  0.608])
b = tf.constant([0.16756377, 0.00662658, 0.54])

In [9]:
sess.run(tf.reduce_sum(
    tf.square(a - b)
))

0.010074299

In [12]:
xyz = occnet_transformer.unproject(uvz)[:, :, :3]
num_kp = tf.shape(xyz)[1]

t1 = tf.tile(xyz, [1, num_kp, 1])
t2 = tf.reshape(
    tf.tile(xyz, [1, 1, num_kp]), tf.shape(t1)
)
t1_np, t2_np = sess.run([t1, t2])
diffsq = tf.square(t1 - t2)

In [14]:
sess.run(diffsq).shape

(1, 10000, 3)

In [15]:
lensqr = tf.reduce_sum(diffsq, axis=2)

In [16]:
sess.run(lensqr).shape

(1, 10000)

In [17]:
sess.run(lensqr)

array([[0.        , 0.00010204, 0.00040814, ..., 0.00040814, 0.00010204,
        0.        ]], dtype=float32)

In [20]:
redsum = tf.reduce_sum(tf.maximum(-lensqr + 0.002, 0.0))

In [21]:
sess.run(redsum)

1.1681968